## Rice Type Classification

This is a set of data created for rice classification. 

### Content
What's inside is more than just rows and columns. You can see rice details listed as column names.

### Description
All attributes are numeric variables and they are listed bellow:

- id
- Area
- MajorAxisLength
- MinorAxisLength
- Eccentricity
- ConvexArea
- EquivDiameter
- Extent
- Perimeter
- Roundness
- AspectRation
- Class

#### Libraries

In [93]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## Data Loading

In [94]:
##Locally loading
df = pd.read_csv('../Datasets/riceClassification.csv')
df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [95]:
#Another option to load the dataset
# import opendatasets as od
# od.download("https://www.kag/gle.com/datasets/mssmartypants/rice-type-classification")

### Data Exploration

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18185 entries, 0 to 18184
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               18185 non-null  int64  
 1   Area             18185 non-null  int64  
 2   MajorAxisLength  18185 non-null  float64
 3   MinorAxisLength  18185 non-null  float64
 4   Eccentricity     18185 non-null  float64
 5   ConvexArea       18185 non-null  int64  
 6   EquivDiameter    18185 non-null  float64
 7   Extent           18185 non-null  float64
 8   Perimeter        18185 non-null  float64
 9   Roundness        18185 non-null  float64
 10  AspectRation     18185 non-null  float64
 11  Class            18185 non-null  int64  
dtypes: float64(8), int64(4)
memory usage: 1.7 MB


In [97]:
# ## droping the id column and missing values
# df=df.drop(columns=['id'])
# df=df.dropna()
df.dropna(inplace=True)
df.drop(['id'],axis=1,inplace=True)
df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18185 entries, 0 to 18184
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Area             18185 non-null  int64  
 1   MajorAxisLength  18185 non-null  float64
 2   MinorAxisLength  18185 non-null  float64
 3   Eccentricity     18185 non-null  float64
 4   ConvexArea       18185 non-null  int64  
 5   EquivDiameter    18185 non-null  float64
 6   Extent           18185 non-null  float64
 7   Perimeter        18185 non-null  float64
 8   Roundness        18185 non-null  float64
 9   AspectRation     18185 non-null  float64
 10  Class            18185 non-null  int64  
dtypes: float64(8), int64(3)
memory usage: 1.5 MB


In [99]:
df.shape

(18185, 11)

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18185 entries, 0 to 18184
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Area             18185 non-null  int64  
 1   MajorAxisLength  18185 non-null  float64
 2   MinorAxisLength  18185 non-null  float64
 3   Eccentricity     18185 non-null  float64
 4   ConvexArea       18185 non-null  int64  
 5   EquivDiameter    18185 non-null  float64
 6   Extent           18185 non-null  float64
 7   Perimeter        18185 non-null  float64
 8   Roundness        18185 non-null  float64
 9   AspectRation     18185 non-null  float64
 10  Class            18185 non-null  int64  
dtypes: float64(8), int64(3)
memory usage: 1.5 MB


In [101]:
df.describe()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
count,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000
mean,7036.492989,151.680754,59.807851,0.915406,7225.817872,94.132952,0.616653,351.606949,0.707998,2.599081,0.549079
std,1467.197150,12.376402,10.061653,0.030575,1502.006571,9.906250,0.104389,29.500620,0.067310,0.434836,0.497599
min,2522.000000,74.133114,34.409894,0.676647,2579.000000,56.666658,0.383239,197.015000,0.174590,1.358128,0.000000
25%,5962.000000,145.675910,51.393151,0.891617,6125.000000,87.126656,0.538530,333.990000,0.650962,2.208527,0.000000
50%,6660.000000,153.883750,55.724288,0.923259,6843.000000,92.085696,0.601194,353.088000,0.701941,2.602966,1.000000
75%,8423.000000,160.056214,70.156593,0.941372,8645.000000,103.559146,0.695664,373.003000,0.769280,2.964101,1.000000
max,10210.000000,183.211434,82.550762,0.966774,11008.000000,114.016559,0.886573,508.511000,0.904748,3.911845,1.000000


In [102]:
df.isnull().sum()

Area               0
MajorAxisLength    0
MinorAxisLength    0
Eccentricity       0
ConvexArea         0
EquivDiameter      0
Extent             0
Perimeter          0
Roundness          0
AspectRation       0
Class              0
dtype: int64

In [103]:
df.duplicated().sum()

np.int64(0)

In [104]:
df.drop_duplicates(inplace=True)

In [105]:
df['Class'].value_counts()

Class
1    9985
0    8200
Name: count, dtype: int64

In [106]:
df['Class'].unique()

array([1, 0])

### Data Preprocessing
- Normalization (to get a value in a certain range)

In [107]:
original_data=df.copy() ## Save our original data for inference

In [108]:
## Divide each column by its maximum value
for column in df.columns:
    df[column]=df[column]/df[column].abs().max()
df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


### Divide dataset into train & test split

In [109]:
X=df.iloc[:,:-1].values
Y=df.iloc[:,-1].values


In [110]:
## Splitting the data into training and testing sets
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

In [111]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(12729, 10)
(5456, 10)
(12729,)
(5456,)


In [112]:
## Now divide the testing data into 15 & 15 testing and validation
X_test,X_val,y_test,y_val=train_test_split(x_test,y_test,test_size=0.5,random_state=42)

In [113]:
print(X_test.shape)
print(X_val.shape)
print(y_test.shape)
print(y_val.shape)

(2728, 10)
(2728, 10)
(2728,)
(2728,)


### Using pytorch dataset and loader class

In [114]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
## dataset class from pytorch

class dataset(Dataset):
    def __init__(self,X,Y):
        self.X=torch.tensor(X,dtype=torch.float32).to(device)
        self.Y=torch.tensor(Y,dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self,idx):
        return self.X[idx],self.Y[idx]

### Create object for training, validation and testing

In [118]:
training_data=dataset(x_train,y_train)
validation_data=dataset(X_val,y_val)
testing_data=dataset(x_test,y_test)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### Creating a dataloadet class
- load the dataset and loop over the entire data

In [ ]:
train_dataloader=DataLoader(training_data,batch_size=10,shuffle=True)
validation_dataloader=DataLoader(validation_data,batch_size=10,shuffle=True)
testing_dataloader=DataLoader(testing_data,batch_size=10,shuffle=True)

### Create a Model

In [ ]:
# class MyModel(nn.Module):
#     def __init__(self,input_size,hidden_size,output_size):
#         super(MyModel,self).__init__()
#         self.fc1=nn.Linear(input_size,hidden_size)
#         self.fc2=nn.Linear(hidden_size,output_size)
#     def forward(self,x):
#         x=torch.relu(self.fc1(x))
#         x=self.fc2(x)
#         return x

# model=MyModel(input_size=11,hidden_size=10,output_size=3)
# model.to(device)

# criterion=nn.CrossEntropyLoss()
# optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

# epochs=100

In [ ]:
HIDDEN_NEURONS=10
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel,self).__init__()
        self.input_Layer=nn.Linear(10,HIDDEN_NEURONS)
        self.linear=nn.Linear(HIDDEN_NEURONS,1)
        self.sigmoid=nn.Sigmoid()
    def forward(self,x):
        x=torch.relu(self.input_Layer(x))
        x=self.linear(x)
        x=self.sigmoid(x)
        return x

model=MyModel().to(device)
# HIDDEN_NEURONS = 10
# class MyModel(nn.Module):
#     def __init__(self):
#         super(MyModel, self).__init__()
#         self.input_Layer = nn.Linear(10, HIDDEN_NEURONS)  # 10 features
#         self.linear = nn.Linear(HIDDEN_NEURONS, 3)  # 3 classes
#         self.softmax = nn.Softmax(dim=1)
#     def forward(self, x):
#         x = torch.relu(self.input_Layer(x))
#         x = self.linear(x)
#         x = self.softmax(x)
#         return x

# model = MyModel().to(device)
# criterion = nn.CrossEntropyLoss()  # For multi-class
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
summary(model,(X.shape[1],))

In [ ]:
# criterion=nn.BCELoss()
# optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
total_loss_train_plot=[]
total_loss_validation_plot=[]
total_accuracy_train_plot=[]
total_accuracy_validation_plot=[]

In [ ]:
# #Training the model
# EPOCHS=10
# for epoch in range(EPOCHS):
#     total_loss_train=0
#     total_acc_train=0
#     total_loss_val=0
#     total_accuracy_val=0
#     for data in train_dataloader:
#         input,label=data
#         output=model(input)

#         prediction=model(input).squeeze(1) ##squeeze is used to remove the extra dimension
#         batch_loss=criterion(prediction,label)
#         total_loss_train+=batch_loss.item()
#         accuracy=((prediction).round()==label).sum().item()
#         # print(accuracy)
#         total_acc_train+=accuracy
#         batch_loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()


In [ ]:
# ### Validation and Testing
# with torch.no_grad(): ##no_grad is used to disable gradient calculation to use model only for testing without training
#     for data in validation_dataloader:
#         input,label=data
#         prediction=model(input).squeeze(1)
#         batch_loss=criterion(prediction,label)
#         total_loss_val+=batch_loss.item()
#         accuracy=((prediction).round()==label).sum().item()
#         total_accuracy_val+=accuracy
#         total_loss_train_plot.append(round(total_loss_train/1000,4))
#         total_accuracy_train_plot.append(round(total_loss_val/1000,4))
#         total_accuracy_train_plot.append(round(total_acc_train/training_data.__len__()*100,4))
#         total_accuracy_validation_plot.append(round(total_accuracy_val/validation_data.__len__()*100,4))
#     print(f"Epoch {epoch+1}/{EPOCHS}, Training Loss: {total_loss_train_plot[-1]:.4f}, Training Accuracy: {total_accuracy_train_plot[-1]:.4f}, Validation Loss: {total_loss_validation_plot[-1]:.4f}, Validation Accuracy: {total_accuracy_validation_plot[-1]:.4f}")

In [ ]:
EPOCHS = 20

for epoch in range(EPOCHS):
    # Training
    model.train()
    total_loss_train = 0
    total_acc_train = 0
    
    for batch_X, batch_y in train_dataloader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        total_loss_train += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_acc_train += (predicted == batch_y).sum().item()
    
    # Validation
    model.eval()
    total_loss_val = 0
    total_accuracy_val = 0
    
    with torch.no_grad():
        for batch_X, batch_y in validation_dataloader:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            total_loss_val += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total_accuracy_val += (predicted == batch_y).sum().item()
    
    # Store metrics
    total_loss_train_plot.append(round(total_loss_train/len(train_dataloader), 4))
    total_accuracy_train_plot.append(round(total_acc_train/training_data.__len__()*100, 4))
    total_loss_validation_plot.append(round(total_loss_val/len(validation_dataloader), 4))
    total_accuracy_validation_plot.append(round(total_accuracy_val/validation_data.__len__()*100, 4))
    
    print(f"Epoch {epoch+1}/{EPOCHS}, Training Loss: {total_loss_train_plot[-1]:.4f}, Training Accuracy: {total_accuracy_train_plot[-1]:.4f}, Validation Loss: {total_loss_validation_plot[-1]:.4f}, Validation Accuracy: {total_accuracy_validation_plot[-1]:.4f}")

In [ ]:
with torch.no_grad():
    total_loss_test = 0.0
    total_accuracy_test = 0

    for inputs, labels in testing_dataloader:
        predictions = model(inputs).squeeze(1)
        batch_loss = criterion(predictions, labels).item()
        total_loss_test += batch_loss
        total_accuracy_test += (predictions.round() == labels).sum().item()
print("Accuracy of the model is: ",round(total_accuracy_test/testing_data.__len__()*100,4))

In [ ]:
fig ,axis=plt.subplots(nrows=1,ncols=2,figsize=(10,10))
axis[0].plot(total_loss_train_plot,label="Training Loss")
axis[0].plot(total_loss_validation_plot,label="Validation Loss")
axis[0].set_title("Loss")
axis[0].legend()
axis[1].plot(total_accuracy_train_plot,label="Training Accuracy")
axis[1].plot(total_accuracy_validation_plot,label="Validation Accuracy")
axis[1].set_title("Accuracy")
axis[1].legend()
plt.show()

axis[1].plot(total_accuracy_train_plot,label="Training Accuracy")
axis[1].plot(total_accuracy_validation_plot,label="Validation Accuracy")
axis[1].set_title("Accuracy")
axis[1].legend()
axis[1].set_xlabel("Epochs")
axis[1].set_ylabel("Accuracy")
axis[1].grid(True)
axis[1].set_ylim(0,100)
axis[1].set_xlim(0,EPOCHS)
axis[1].set_xticks(np.arange(0,EPOCHS+1,1))
axis[1].set_yticks(np.arange(0,101,10))
axis[1].set_xticklabels(np.arange(0,EPOCHS+1,1))
axis[1].set_yticklabels(np.arange(0,101,10))
axis[1].set_xticklabels(np.arange(0,EPOCHS+1,1))

### Testing

In [ ]:
area=2332/original_data['Area'].abs().sum()
MajorAxisLength=233/original_data['MajorAxisLength'].abs().sum()
MinorAxisLength=332/original_data['MinorAxisLength'].abs().sum()
Eccentricity=32/original_data['Eccentricity'].abs().sum()
ConvexArea=32/original_data['ConvexArea'].abs().sum()
EquivDiameter=2332/original_data['EquivDiameter'].abs().sum()
Roundness=22/original_data['Roundness'].abs().sum()
Extent=2/original_data['Extent'].abs().sum()
Perimeter=332/original_data['Perimeter'].abs().sum()
AspectRation=52/original_data['AspectRation'].abs().sum()

In [ ]:
df.head()

In [ ]:
my_pred=model(torch.tensor([area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Roundness,Extent,Perimeter,AspectRation],dtype=torch.float32).to(device))

In [ ]:
my_pred